In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import scipy.io as io

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [2]:
PATH = 'ml/results_rf_030822_rich_region.csv'

In [3]:
results = pd.read_csv(PATH, index_col=0)
complete = results[results.restriction == 'complete']
complete = complete.dropna(how='all', axis=1)
grouped_complete = complete.groupby(by=['restriction', 'modality', 'chunk_size'])
grouped_complete[['auc', 'dev_auc']].quantile(q=[0.25, 0.5, 0.75])

auc   dev_auc
restriction modality      chunk_size                         
complete    morphological 0.0        0.25  0.981237  0.987782
                                     0.50  0.989858  0.995301
                                     0.75  0.993281  1.000000
                          25.0       0.25  0.980350  0.990602
                                     0.50  0.988337  0.998120
                                     0.75  0.994929  1.000000
                          50.0       0.25  0.978195  0.986137
                                     0.50  0.980984  0.997650
                                     0.75  0.987576  1.000000
                          1600.0     0.25  0.973631  0.993656
                                     0.50  0.976673  0.998120
                                     0.75  0.979209  1.000000
            spatial       0.0        0.25  0.577459  0.794760
                                     0.50  0.622211  0.821194
                                     0.75  0.638692  0.867951
                          25.0       0.25  0.805274  0.932331
                                     0.50  0.829615  0.967575
                                     0.75  0.846856  0.979323
                          50.0       0.25  0.735294  0.938792
                                     0.50  0.772312  0.960056
                                     0.75  0.795132  0.980263
                          1600.0     0.25  0.695106  0.901316
                                     0.50  0.709432  0.926457
                                     0.75  0.719954  0.954887
            temporal      0.0        0.25  0.762677  0.977444
                                     0.50  0.774594  0.990602
                                     0.75  0.787145  0.995301
                          25.0       0.25  0.654792  0.957589
                                     0.50  0.675710  0.988722
                                     0.75  0.705629  0.993186
                          50.0       0.25  0.715644  0.973097
                                     0.50  0.742140  0.988722
                                     0.75  0.760015  0.994831
                          1600.0     0.25  0.776116  0.977679
                                     0.50  0.791329  0.990602
                                     0.75  0.810091  0.994361

In [4]:
df0 = complete[complete.chunk_size == 0]
wf_ca1 = complete[complete.modality == 'morphological'].dev_auc.to_numpy()
wf_ncx = complete[complete.modality == 'morphological'].auc.to_numpy()

temp_ca1 = complete[complete.modality == 'temporal'].dev_auc.to_numpy()
temp_ncx = complete[complete.modality == 'temporal'].auc.to_numpy()

spat_ca1 = complete[complete.modality == 'spatial'].dev_auc.to_numpy()
spat_ncx = complete[complete.modality == 'spatial'].auc.to_numpy()

statistic, p_val = stats.mannwhitneyu(wf_ca1, wf_ncx)
print(f"Mann-Whitney statistical test results for WF are p-value={p_val} (statistic={statistic})")
print()

statistic, p_val = stats.mannwhitneyu(temp_ca1, temp_ncx)
print(f"Mann-Whitney statistical test results for temporal are p-value={p_val} (statistic={statistic})")
print()

statistic, p_val = stats.mannwhitneyu(spat_ca1, spat_ncx)
print(f"Mann-Whitney statistical test results for spatial are p-value={p_val} (statistic={statistic})")
print()

Mann-Whitney statistical test results for WF are p-value=8.328078547901332e-20 (statistic=9576.5)

Mann-Whitney statistical test results for temporal are p-value=2.3808319237524272e-67 (statistic=0.0)

Mann-Whitney statistical test results for spatial are p-value=2.5774582291550415e-53 (statistic=2281.0)



In [5]:
cs_50 = complete[complete.chunk_size == 50]
cs_wf = cs_50[cs_50.modality == 'morphological']
wf_aucs = cs_wf.auc.to_numpy()
wf_dev_aucs = cs_wf.dev_auc.to_numpy()

cs_1600 = complete[complete.chunk_size == 1600]
cs_temp = cs_1600[cs_1600.modality == 'temporal']
temp_aucs = cs_temp.auc.to_numpy()
temp_dev_aucs = cs_temp.dev_auc.to_numpy()

cs_25 = complete[complete.chunk_size == 25]
cs_spat = cs_25[cs_25.modality == 'spatial']
spat_aucs = cs_spat.auc.to_numpy()
spat_dev_aucs = cs_spat.dev_auc.to_numpy()

aucs = np.concatenate((wf_aucs, temp_aucs, spat_aucs))
dev_aucs = np.concatenate((wf_dev_aucs, temp_dev_aucs, spat_dev_aucs))
N = len(aucs)

all_aucs = np.concatenate((aucs, dev_aucs))
g1 = np.concatenate((np.ones(N), 2 * np.ones(N)))
g2 = np.repeat([[1, 2, 3, 1, 2, 3]], N // 3, axis=1).flatten()

In [6]:
m_dict = {'data': all_aucs, 'g1': g1, 'g2': g2}
io.savemat('region_mat.mat', m_dict)

In [7]:
wf_diff = 100 * (wf_dev_aucs - wf_aucs) / wf_dev_aucs
temp_diff = 100 * (temp_dev_aucs - temp_aucs) / temp_dev_aucs
spat_diff = 100 * (spat_dev_aucs - spat_aucs) / spat_dev_aucs

wf_median, wf_prec25, wf_prec75 = np.percentile(wf_diff, [50, 25, 75])
print(f"Median of WF column is {wf_median: .3f} [{wf_prec25: .3f}, {wf_prec75: .3f}]")
temp_median, temp_prec25, temp_prec75 = np.percentile(temp_diff, [50, 25, 75])
print(f"Median of temporal column is {temp_median: .3f} [{temp_prec25: .3f}, {temp_prec75: .3f}]")
spat_median, spat_prec25, spat_prec75 = np.percentile(spat_diff, [50, 25, 75])
print(f"Median of spatial column is {spat_median: .3f} [{spat_prec25: .3f}, {spat_prec75: .3f}]\n")

statistic, p_val = stats.wilcoxon(wf_dev_aucs, wf_aucs, alternative='greater')
print(f"Wilcoxon statistical test results for wf are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_dev_aucs, spat_aucs, alternative='greater')
print(f"Wilcoxon statistical test results for spatial are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_dev_aucs, temp_aucs, alternative='greater')
print(f"Wilcoxon statistical test results for temporal p-value={p_val} (statistic={statistic})\n")

statistic, p_val = stats.wilcoxon(wf_diff, temp_diff)
print(f"Wilcoxon statistical test results for wf and temp are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(wf_diff, spat_diff)
print(f"Wilcoxon statistical test results for wf and spatial are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_diff, temp_diff)
print(f"Wilcoxon statistical test results for temporal and spatial p-value={p_val} (statistic={statistic})")

Median of WF column is  1.150 [ 0.517,  1.799]
Median of temporal column is  19.574 [ 17.171,  21.133]
Median of spatial column is  13.070 [ 9.638,  16.594]

Wilcoxon statistical test results for wf are p-value=0.005637717584190368 (statistic=900.0)
Wilcoxon statistical test results for spatial are p-value=3.778464727931783e-10 (statistic=1275.0)
Wilcoxon statistical test results for temporal p-value=3.778464727931783e-10 (statistic=1275.0)

Wilcoxon statistical test results for wf and temp are p-value=7.556929455863566e-10 (statistic=0.0)
Wilcoxon statistical test results for wf and spatial are p-value=7.556929455863566e-10 (statistic=0.0)
Wilcoxon statistical test results for temporal and spatial p-value=8.212511505089428e-08 (statistic=82.0)


In [8]:
statistic, p_val = stats.wilcoxon(wf_aucs - 0.5)
print(f"Mann-Whitney statistical test results for wf ncx auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_aucs - 0.5)
print(f"Mann-Whitney statistical test results for spatial ncx auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_aucs - 0.5)
print(f"Mann-Whitney statistical test results for temporal ncx auc are p-value={p_val} (statistic={statistic})")

statistic, p_val = stats.wilcoxon(wf_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for wf ca1 auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for spatial ca1 auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for temporal ca1 auc are p-value={p_val} (statistic={statistic})")

Mann-Whitney statistical test results for wf ncx auc are p-value=7.482079377523655e-10 (statistic=0.0)
Mann-Whitney statistical test results for spatial ncx auc are p-value=7.541564726440781e-10 (statistic=0.0)
Mann-Whitney statistical test results for temporal ncx auc are p-value=7.553512643873762e-10 (statistic=0.0)
Mann-Whitney statistical test results for wf ca1 auc are p-value=6.436453149724364e-10 (statistic=0.0)
Mann-Whitney statistical test results for spatial ca1 auc are p-value=7.541564726440781e-10 (statistic=0.0)
Mann-Whitney statistical test results for temporal ca1 auc are p-value=7.441533360918728e-10 (statistic=0.0)


In [9]:
statistic, p_val = stats.wilcoxon(temp_aucs, spat_aucs)
print(f"Wilcoxon statistical test results for spatial and temporal are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_aucs, wf_aucs)
print(f"Wilcoxon statistical test results for wf and temporal are p-value={p_val} (statistic={statistic})")

Wilcoxon statistical test results for spatial and temporal are p-value=1.0259748838649458e-05 (statistic=180.5)
Wilcoxon statistical test results for wf and temporal are p-value=7.553512643873762e-10 (statistic=0.0)
